<b>
<div style="text-align: center;">
<span style="font-size: 200%;">状態価値関数 (State Value Function)</span>
</div>
</b>

<br>

<b>
<div style="text-align: right;">
    <span style="font-size: 150%;">
        2019/12/20 
    </span><br>
    <span style="font-size: 150%;">
        Masaya Mori
    </span>
</div>
</b>

# 問題設定

![問題設定](figure/ProblemSetting.jpg)

- <span style="font-size: 120%;"> 今回は5×5の格子世界(Grid World)における各マス目の状態価値を求める</span>
- <span style="font-size: 120%;"> エージェントである棒人間は前後左右に動くことができる．また今回はスタート地点を考えないものとする．つまり，現在エージェントは(1,2)にいるが，ここがスタートという訳ではなく，あるマス目でスタートした時のそのマス目の状態価値を求めるということである</span>
- <span style="font-size: 120%;"> 報酬はポイントマスで何かしらの行動を取った時に得ることができ，報酬を獲得した後は強制的にワープマスへ飛ばされる</span>
- <span style="font-size: 120%;"> 境界線の外側には出れないこととする．仮に外に出た場合，-1ptの報酬とする．またその時の位置は，境界線を出る1つ前の位置に戻されることとする</span>
- <span style="font-size: 120%;"> 状態価値は，
    $${\rm V^{\pi}}(s) = \Sigma_{a}{\rm \pi} (s,a)\Sigma_{s'}{\rm P}^{a}_{s,s'}\bigr[ {\rm R}^{a}_{s,s'} + \gamma {\rm V^{\pi}}(s') \bigl], \nonumber$$
によって算出する</span>

# マルコフ決定過程

- <span style="font-size: 120%;">状態s&isin;S: 現在のマス目(x,y)</span>  
- <span style="font-size: 120%;">行動a&isin;A: エージェントの上下左右の動き { (0,1) (0,-1) (-1,0) (1,0) }</span>  
- <span style="font-size: 120%;">確率遷移T(s,a,s'){ = P(s'|s,a) }: 今回は確定的に"1"とする(右への気分が60%,左への気分は40%みたいのは無し) </span>  
- <span style="font-size: 120%;">報酬R(s,a,s'): </span>
    $$
    {\rm R(s,a,s')} = \left\{
    \begin{array}{rl}
    -1pt & (境界線外へ移動) \\
    5pt & (5pt報酬マスで行動) \\
    10pt & (10pt報酬マスで行動) \\
    0pt & (上記以外)
    \end{array} \nonumber
    \right.
    $$ <br>
- <span style="font-size: 120%;">方策&pi;(s,a): </span>
    $$
    {\rm \pi(s,a)} = \left\{
    \begin{array}{rl}
    0.25 & (any, up) \\
    0.25 & (any, down) \\
    0.25 & (any, right) \\
    0.25 & (any, left)
    \end{array} \nonumber
    \right.
    $$  

<span style="font-size: 120%;"> ※そもそも『マルコフ決定過程って何?』という方は，[こちら](https://www.hellocybernetics.tech/entry/2019/06/24/192311)を読んでください． </span>

# Pythonでの実装

## ~~とりあえず実装してみる~~ ⇒ 3.4(ソースコードの修正)へGO!

In [49]:
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm
import time

In [50]:
stage = [[0,4],[1,4],[2,4],[3,4],[4,4],
         [0,3],[1,3],[2,3],[3,3],[4,3],
         [0,2],[1,2],[2,2],[3,2],[4,2],
         [0,1],[1,1],[2,1],[3,1],[4,1],
         [0,0],[1,0],[2,0],[3,0],[4,0]]

In [58]:
class Agent(): # stage_map, the array number of the stage    
    
    def __init__(self):
        # 行動a,方策πの宣言。引数で使うので、初めに宣言する。
        self.actions = [[0,1],[0,-1],[-1,0],[1,0]] # up,down,left,right
        self.each_pi = [0.25,0.25,0.25,0.25]
        self.position = []
    
    def set_pos(self,now_pos):
        # 現在地の更新
        self.position = now_pos
        
    def get_pos(self):
        # 現在地の取得
        return self.position
    
    def move(self,action):
        # 行動後の位置の取得
        if self.get_pos() == [1,4]: # 10ptゾーン
            next_pos = [1,0]
        elif self.get_pos() == [3,4]: # 5ptゾーン
            next_pos = [3,2]
        else :
            next_pos = [self.get_pos()[0] + action[0], self.get_pos()[1] + action[1]]
        
        # 境界線外に出ている時の処理
        if 0 > self.get_pos()[0] or self.get_pos()[0] > 4 or 0 > self.get_pos()[1] or self.get_pos()[1] > 4:
            next_pos = self.get_pos()
            
        self.set_pos(next_pos)
    
    def pi(self,state,num):
        return self.each_pi[num]
        
    def reward(self,state,action):
        if state == [1,4]:
            return 10
        
        if state == [3,4]:
            return 5
        
        if state[1] == 4 and action == [0,1]:
            return -1
        elif state[1] == 0 and action == [0,-1]:
            return -1
        elif state[0] == 0 and action == [-1,0]:
            return -1
        elif state[0] == 4 and action == [1,0]:
            return -1
        else :
            return 0
    
    def v_pi(self,state,n,out,iter_num): # この"state"は，潜った時の位置を記憶しておくため
        if n == iter_num :
            for i, action in enumerate(self.actions):
                # 最下層での処理．最下層では前後左右に動いた際の報酬を一気に計算する．
                out += self.pi(state,i) * self.reward(state,action)
            return out
        else :
            for i, action in enumerate(self.actions):
                # 先に π*R を計算し，action方向に移動する
                out += self.pi(self.get_pos(),i) * self.reward(self.get_pos(),action)
                self.move(action)
                # この時、"state"は1つ前の位置，"self.get_pos()"には現在の位置が記憶されている
                # 移動後，つまり現在の状態はs'となっているので，s'の報酬を計算する
                out += self.pi(self.get_pos(),i) * 0.9 * self.v_pi(self.get_pos(), n+1, 0, iter_num)
                # ここで再帰関数を使っているので，"n"が"iter_num"と同じ回数
                # つまり，指定した深さまで潜らないと次のプログラムへ進むことはできない
                self.set_pos(state) # "self.get_pos()"から1つ前の状態"state"に戻るため
            return out

In [52]:
agent = Agent()
v_pi_value = []

start = time.time()
for i,locate in tqdm(enumerate(stage)):
    agent.set_pos(locate)
    v_pi_value.append(agent.v_pi(agent.get_pos(),0,0,11))
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")


elapsed_time:945.1679182052612[sec]


In [26]:
outcome = np.array(v_pi_value)
outcome = outcome.reshape(5,5)
outcome

array([[-0.5 , 10.  , -0.25,  5.  , -0.5 ],
       [-0.25,  0.  ,  0.  ,  0.  , -0.25],
       [-0.25,  0.  ,  0.  ,  0.  , -0.25],
       [-0.25,  0.  ,  0.  ,  0.  , -0.25],
       [-0.5 , -0.25, -0.25, -0.25, -0.5 ]])

## 結果・考察

![実行結果](figure/outcome.jpg)

- <span style="font-size: 120%;"> 全体的にスコアが低い気がする</span>  
<span style="font-size: 120%;"> ⇒ 報酬に関する記述間違えてる?</span>  
- <span style="font-size: 120%;"> 値は間違えてるけど、10ptや5pt付近のエリアの状態価値が高くなっている</span>  
<span style="font-size: 120%;"> ⇒ 大まかには合ってそう．少なくとも再帰は合ってるはず</span>  
- <span style="font-size: 120%;"> 実行時間がやばい(再帰11回で15分強)。</span>  
<span style="font-size: 120%;"> ⇒ 再帰+for文は時間がかかる(動的計画法の問題点)</span>

## 1つずつ確認していこう!!!

<span style="font-size: 120%;"> 初めに，再帰無しの場合を考える． </span>

In [53]:
agent = Agent()
v_pi_value = []
for i,locate in enumerate(stage):
    agent.set_pos(locate)
    v_pi_value.append(agent.v_pi(agent.get_pos(),0,0,0))
outcome = np.array(v_pi_value)
outcome = outcome.reshape(5,5)
outcome

array([[-0.5 , 10.  , -0.25,  5.  , -0.5 ],
       [-0.25,  0.  ,  0.  ,  0.  , -0.25],
       [-0.25,  0.  ,  0.  ,  0.  , -0.25],
       [-0.25,  0.  ,  0.  ,  0.  , -0.25],
       [-0.5 , -0.25, -0.25, -0.25, -0.5 ]])

<span style="font-size: 120%;">(0,4) と(1,4)の価値関数を計算をしてみると，</span> </br>

$$
\begin{align}
    0.25 \times -1 + 0.25 \times  0 + 0.25 \times -1 + 0.25 \times  0 &=& -0.5, \\
    0.25 \times 10 + 0.25 \times 10 + 0.25 \times 10 + 0.25 \times 10 &=&   10
\end{align}
$$

<span style="font-size: 120%;">となり，合っていることがわかる(計算順は上下左右)．</span>

<span style="font-size: 120%;">続いて，再帰1回の場合を考える．</span>

In [54]:
agent = Agent()
v_pi_value = []
for i,locate in enumerate(stage):
    agent.set_pos(locate)
    v_pi_value.append(agent.v_pi(agent.get_pos(),0,0,1))
outcome = np.array(v_pi_value)
outcome = outcome.reshape(5,5)
outcome

array([[ 1.58125,  9.775  ,  3.125  ,  5.     ,  0.45625],
       [-0.41875,  2.19375, -0.05625,  1.06875, -0.41875],
       [-0.3625 , -0.05625,  0.     , -0.05625, -0.3625 ],
       [-0.41875, -0.1125 , -0.05625, -0.1125 , -0.41875],
       [-0.725  , -0.41875, -0.3625 , -0.41875, -0.725  ]])

<span style="font-size: 120%;">(0,4) と(1,4)の価値関数を計算をしてみると，</span> </br>

$$
\begin{split}
& 0.25 \times \bigl\{ -1 + 0.9\times(0.25 \times -1 + 0.25 \times  \;\,0 + 0.25 \times -1 + 0.25 \times \;\,0 ) \bigr\} \\
& 0.25 \times \bigl\{\;\,\,0 + 0.9\times(0.25 \times \;\,\,0 + 0.25 \times  \;\,0 + 0.25 \times -1 + 0.25 \times \;\,0 ) \bigr\} \\
& 0.25 \times \bigl\{ -1 + 0.9\times(0.25 \times -1 + 0.25 \times  \;\,0 + 0.25 \times -1 + 0.25 \times \;\,0 ) \bigr\} \\
& 0.25 \times \bigl\{\;\,\,0 + 0.9\times(0.25 \times \,10 + 0.25 \times 10 + 0.25 \times \,10 + 0.25 \times 10 ) \bigr\} = 1.46875
\end{split} \nonumber
$$

<span style="font-size: 120%;">となり，間違っていることが分かる(計算順は上下左右)．</span>

In [55]:
0.25*(-1 + 0.9 *(0.25*-1 + 0.25*0 + 0.25*-1 + 0.25*0)) + \
0.25*(0 + 0.9 *(0.25*0 + 0.25*0 + 0.25*-1 + 0.25*0)) + \
0.25*(-1 + 0.9 *(0.25*-1 + 0.25*0 + 0.25*-1 + 0.25*0)) + \
0.25*(0 + 0.9 *(0.25*10 + 0.25*10 + 0.25*10 + 0.25*10)) 

1.46875

<span style="font-size: 120%;">ここで，上記の式の1行目または3行目における再帰箇所の"-1"を全て"0"に置き換えると，</span>

In [57]:
0.25*(-1 + 0.9 *(0.25*-1 + 0.25*0 + 0.25*-1 + 0.25*0)) + \
0.25*(0 + 0.9 *(0.25*0 + 0.25*0 + 0.25*-1 + 0.25*0)) + \
0.25*(-1 + 0.9 *(0.25*0 + 0.25*0 + 0.25*0 + 0.25*0)) + \
0.25*(0 + 0.9 *(0.25*10 + 0.25*10 + 0.25*10 + 0.25*10)) 

1.58125

<span style="font-size: 120%;"> となり，(0,4)の状態価値と値が同じになる． </span>

<span style="font-size: 120%;">これらのことから，境界線を超えた時に対する処理の記述が間違えているということが分かる．</span>

## ソースコードの修正

In [39]:
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm
import time

In [40]:
stage = [[0,4],[1,4],[2,4],[3,4],[4,4],
         [0,3],[1,3],[2,3],[3,3],[4,3],
         [0,2],[1,2],[2,2],[3,2],[4,2],
         [0,1],[1,1],[2,1],[3,1],[4,1],
         [0,0],[1,0],[2,0],[3,0],[4,0]]

In [41]:
class Agent(): # stage_map, the array number of the stage    
    
    def __init__(self):
        # 行動a,方策πの宣言。引数で使うので、初めに宣言する。
        self.actions = [[0,1],[0,-1],[-1,0],[1,0]] # up,down,left,right
        self.each_pi = [0.25,0.25,0.25,0.25]
        self.position = []
    
    def set_pos(self,now_pos):
        # 現在地の更新
        self.position = now_pos
        
    def get_pos(self):
        # 現在地の取得
        return self.position
    
    def move(self,action):
        # 行動後の位置の取得
        if self.get_pos() == [1,4]: # 10ptゾーン
            next_pos = [1,0]
        elif self.get_pos() == [3,4]: # 5ptゾーン
            next_pos = [3,2]
        else :
            next_pos = [self.get_pos()[0] + action[0], self.get_pos()[1] + action[1]]
        
        # 境界線外に出ている時の処理
        if 0 > next_pos[0] or next_pos[0] > 4 or 0 > next_pos[1] or next_pos[1] > 4:
            next_pos = self.get_pos()
            
        self.set_pos(next_pos)
    
    def pi(self,state,num):
        return self.each_pi[num]
        
    def reward(self,state,action):
        if state == [1,4]:
            return 10
        
        if state == [3,4]:
            return 5
        
        if state[1] == 4 and action == [0,1]:
            return -1
        elif state[1] == 0 and action == [0,-1]:
            return -1
        elif state[0] == 0 and action == [-1,0]:
            return -1
        elif state[0] == 4 and action == [1,0]:
            return -1
        else :
            return 0
    
    def v_pi(self,state,n,out,iter_num): # この"state"は，潜った時の位置を記憶しておくため
        if n == iter_num :
            for i, action in enumerate(self.actions):
                # 最下層での処理．最下層では前後左右に動いた際の報酬を一気に計算する．
                out += self.pi(state,i) * self.reward(state,action)
            return out
        else :
            for i, action in enumerate(self.actions):
                # 先に π*R を計算し，action方向に移動する
                out += self.pi(self.get_pos(),i) * self.reward(self.get_pos(),action)
                self.move(action)
                # この時、"state"は1つ前の位置，"self.get_pos()"には現在の位置が記憶されている
                # 移動後，つまり現在の状態はs'となっているので，s'の報酬を計算する
                out += self.pi(self.get_pos(),i) * 0.9 * self.v_pi(self.get_pos(), n+1, 0, iter_num)
                # ここで再帰関数を使っているので，"n"が"iter_num"と同じ回数
                # つまり，指定した深さまで潜らないと次のプログラムへ進むことはできない
                self.set_pos(state) # "self.get_pos()"から1つ前の状態"state"に戻るため
            return out

In [47]:
agent = Agent()
v_pi_value = []

start = time.time()
for i,locate in tqdm(enumerate(stage)):
    agent.set_pos(locate)
    v_pi_value.append(agent.v_pi(agent.get_pos(),0,0,11))
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")


elapsed_time:816.9026861190796[sec]


In [48]:
outcome = np.array(v_pi_value)
outcome = outcome.reshape(5,5)
outcome

array([[ 3.34372942,  8.86087598,  4.44351842,  5.31948271,  1.43977865],
       [ 1.5304277 ,  3.01166897,  2.24619831,  1.88552175,  0.50419075],
       [ 0.05982435,  0.74571609,  0.68446781,  0.354529  , -0.40967115],
       [-0.94315215, -0.39541655, -0.3191193 , -0.53994032, -1.1414136 ],
       [-1.80039152, -1.28475124, -1.15821025, -1.34834579, -1.89474334]])

![修正後実行結果](figure/fixed_outcome.jpg)

# まとめ

![まとめ](figure/conclusion.jpg)

<span style="font-size: 120%;"> 今回は状態価値関数を用いた各マス目の状態価値の導出を行った．</span>  
<span style="font-size: 120%;"> 次回は，[行動価値関数を用いた行動価値の導出](https://github.com/rrrrind/reinforcement-learning/blob/master/DP/src/Action%20Value%20Function/Action_Value_Function.ipynb)を行う．</span>

# 参考資料

- [強化学習の基礎：マルコフ決定過程ってなんぞ？](https://www.hellocybernetics.tech/entry/2019/06/24/192311)
- [今さら聞けない強化学習（1）：状態価値関数とBellman方程式](https://qiita.com/triwave33/items/5e13e03d4d76b71bc802)
- [今さら聞けない強化学習（2）：状態価値関数の実装](https://qiita.com/triwave33/items/3bad9f35d213a315ce78)